In [1]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip subdirectories from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH`
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from src import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
Added the following directory to the PYTHONPATH: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


In [2]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from src.airquality import util
import os

warnings.filterwarnings("ignore")

In [24]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

# Optional single sensor mode if ID is set in .env
sensor_id_param = getattr(settings, 'SENSOR_ID', None)

if sensor_id_param:
    # Read one secret for single sensor mode
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id_param}"
    location_str = secrets.get_secret(secret_name).value
    locations = {sensor_id_param: json.loads(location_str)}
else:
    # Read all individual secrets in batch mode
    all_secrets = secrets.get_secrets()
    locations = {}
    for secret in all_secrets:
        if secret.name.startswith("SENSOR_LOCATION_JSON_"):
            sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
            location_str = secrets.get_secret(secret.name).value
            if location_str:
                locations[sensor_id] = json.loads(location_str)


2025-11-16 19:41:34,909 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-16 19:41:34,913 INFO: Initializing external client
2025-11-16 19:41:34,913 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 19:41:36,440 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279179


In [25]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_complete",
    version=1,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2025-11-16 19:41:41,232 INFO: There is no parent information


Downloading: 0.000%|          | 0/524552 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/132786 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/124667 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/73105 elapsed<00:00 remaining<?

...

Downloading: 0.000%|          | 0/124715 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/130422 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/115716 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/116567 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/65753 elapsed<00:00 remaining<?

In [26]:
retrieved_xgboost_model = XGBRegressor()
retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")
model_feature_names = retrieved_xgboost_model.get_booster().feature_names

In [27]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)
weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

today = datetime.datetime.now().replace(tzinfo=None)
batch_data = weather_fg.filter(weather_fg.date >= today).read()
batch_data["date"] = pd.to_datetime(batch_data["date"]).dt.tz_localize(None)
print(batch_data.info())


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         90 non-null     datetime64[us]
 1   temperature_2m_mean          90 non-null     float32       
 2   precipitation_sum            90 non-null     float32       
 3   wind_speed_10m_max           90 non-null     float32       
 4   wind_direction_10m_dominant  90 non-null     float32       
 5   city                         90 non-null     object        
 6   sensor_id                    90 non-null     object        
 7   latitude                     90 non-null     float64       
 8   longitude                    90 non-null     float64       
dtypes: datetime64[us](1), float32(4), float64(2), object(2)
memory usage: 5.1+ KB
None


In [28]:
def add_rolling_window_feature(df, window_days=3, column="pm25", new_column="pm25_rolling_3d"):
    df = df.sort_values(["sensor_id", "date"]).copy()
    df_indexed = df.set_index("date", append=False)
    df_indexed[f"{column}_shifted"] = df_indexed.groupby("sensor_id")[column].shift(1)
    df[new_column] = (
        
        df_indexed.groupby("sensor_id")[f"{column}_shifted"]
        .rolling(window=f"{window_days}D", min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
        .values
    )
    return df

def add_lagged_features(df, column="pm25", lags=[1, 2, 3]):
    df = df.sort_values(["sensor_id", "date"]).copy()
    for lag in lags:
        new_column = f"{column}_lag_{lag}d"
        df[new_column] = df.groupby("sensor_id")[column].shift(lag)
    return df

def predict_for_sensor(sensor_weather, sensor_historical, location, model, model_feature_names):
    combined = pd.concat([
        sensor_historical[["date", "sensor_id", "pm25"]],
        sensor_weather[["date", "sensor_id"]].assign(pm25=None)
    ], ignore_index=True).sort_values(["sensor_id", "date"])
    
    combined = add_rolling_window_feature(combined, window_days=3, column="pm25", new_column="pm25_rolling_3d")
    combined = add_lagged_features(combined, column="pm25", lags=[1, 2, 3])
    
    sensor_weather = sensor_weather.merge(
        combined[["date", "pm25_rolling_3d", "pm25_lag_1d", "pm25_lag_2d", "pm25_lag_3d"]],
        on="date", how="left"
    )
    
    available_features = [col for col in model_feature_names if col in sensor_weather.columns]
    sensor_weather['predicted_pm25'] = model.predict(sensor_weather[available_features])
    sensor_weather['street'] = location['street']
    sensor_weather['city'] = location['city']
    sensor_weather['country'] = location['country']
    return sensor_weather


In [29]:
historical_start = today - datetime.timedelta(days=4)
try:
    historical_pm25 = air_quality_fg.read()
    historical_pm25["date"] = pd.to_datetime(historical_pm25["date"]).dt.tz_localize(None)
    historical_pm25 = historical_pm25[(historical_pm25["date"] >= historical_start) & (historical_pm25["date"] < today)][["date", "sensor_id", "pm25"]]
except:
    historical_pm25 = pd.DataFrame()

# Generate predictions for all sensors
all_predictions = []
for sensor_id, location in locations.items():
    sensor_weather = batch_data[batch_data["sensor_id"] == sensor_id].copy()
    if sensor_weather.empty:
        continue
    sensor_historical = historical_pm25[historical_pm25["sensor_id"] == sensor_id] if not historical_pm25.empty else pd.DataFrame()
    all_predictions.append(predict_for_sensor(sensor_weather, sensor_historical, location, retrieved_xgboost_model, model_feature_names))

batch_data = pd.concat(all_predictions, ignore_index=True).sort_values(['sensor_id', 'date']) if all_predictions else pd.DataFrame()
batch_data['days_before_forecast_day'] = batch_data.groupby('sensor_id').cumcount() + 1
batch_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.61s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,sensor_id,latitude,longitude,pm25_rolling_3d,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d,predicted_pm25,street,country,days_before_forecast_day
75,2025-11-17,1.60,0.0,10.188700,327.994659,Gothenburg,112672,57.660000,12.000000,5.0,5.0,NaN,NaN,4.613566,Bågskyttegatan,Sweden,1
77,2025-11-18,2.55,1.2,5.692099,145.304779,Gothenburg,112672,57.660000,12.000000,5.0,NaN,5.0,NaN,10.450383,Bågskyttegatan,Sweden,2
76,2025-11-19,-0.10,0.0,12.434340,67.890503,Gothenburg,112672,57.660000,12.000000,5.0,NaN,NaN,5.0,1.172788,Bågskyttegatan,Sweden,3
73,2025-11-20,-0.65,0.0,18.875126,34.902569,Gothenburg,112672,57.660000,12.000000,NaN,NaN,NaN,NaN,4.004723,Bågskyttegatan,Sweden,4
72,2025-11-21,-2.30,0.0,5.154416,12.094739,Gothenburg,112672,57.660000,12.000000,NaN,NaN,NaN,NaN,5.830064,Bågskyttegatan,Sweden,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2025-11-18,2.60,1.2,5.692099,145.304779,Gothenburg,88372,57.648728,12.008352,3.0,NaN,3.0,NaN,11.744453,Ridlärargatan,Sweden,2
59,2025-11-19,-0.05,0.0,12.434340,67.890503,Gothenburg,88372,57.648728,12.008352,3.0,NaN,NaN,3.0,1.403218,Ridlärargatan,Sweden,3
55,2025-11-20,-0.60,0.0,18.875126,34.902569,Gothenburg,88372,57.648728,12.008352,NaN,NaN,NaN,NaN,3.933114,Ridlärargatan,Sweden,4
54,2025-11-21,-2.25,0.0,5.154416,12.094739,Gothenburg,88372,57.648728,12.008352,NaN,NaN,NaN,NaN,5.451262,Ridlärargatan,Sweden,5


In [30]:
pred_file_paths = []
for sensor_id, location in locations.items():
    sensor_data = batch_data[batch_data["sensor_id"] == sensor_id].copy()
    if sensor_data.empty:
        continue
    
    city = location['city']
    street = location['street']
    pred_file_path = f"{root_dir}/models/air_quality_complete/images/forecast_{city}_{street}.png"
    plt = util.plot_air_quality_forecast(city, street, sensor_data, pred_file_path)
    plt.close()
    pred_file_paths.append(pred_file_path)

In [31]:
monitor_fg = fs.get_or_create_feature_group(
    name='aq_predictions',
    description='Air Quality prediction monitoring',
    version=1,
    primary_key=['sensor_id','date','days_before_forecast_day'],
    event_time="date"
)

In [32]:
monitor_fg.insert(batch_data, wait=True)

Uploading Dataframe: 100.00% |██████████| Rows 90/90 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/aq_predictions_1_offline_fg_materialization/executions
2025-11-16 19:42:22,480 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-16 19:42:26,185 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-16 19:44:05,512 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-16 19:44:05,966 INFO: Waiting for log aggregation to finish.
2025-11-16 19:44:05,967 INFO: Execution finished successfully.


(Job('aq_predictions_1_offline_fg_materialization', 'SPARK'), None)

In [33]:
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.32s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,sensor_id,latitude,longitude,pm25_rolling_3d,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d,predicted_pm25,street,country,days_before_forecast_day
0,2025-11-17 00:00:00+00:00,1.60,0.0,10.188700,327.994659,Lundby,59893,57.702000,11.904000,3.0,3.0,NaN,NaN,1.423372,Londongatan,Sweden,1
1,2025-11-17 00:00:00+00:00,1.70,0.0,12.768586,338.498505,Lindome,404209,57.601655,12.100873,3.0,3.0,NaN,NaN,1.466268,Högkullevägen,Sweden,1
2,2025-11-17 00:00:00+00:00,1.55,0.0,10.188700,327.994659,Mölndal,59095,57.652000,11.968000,2.0,2.0,NaN,NaN,2.304359,Eklanda Slätt,Sweden,1
3,2025-11-17 00:00:00+00:00,1.60,0.0,10.188700,327.994659,Majorna-Linné,60853,57.698000,11.946000,2.0,2.0,NaN,NaN,1.809121,Masthugget,Sweden,1
4,2025-11-17 00:00:00+00:00,1.40,0.0,10.188700,327.994659,Norra Hisingen,61714,57.750000,11.970000,4.0,4.0,NaN,NaN,2.295515,Nyhemsgatan,Sweden,1
5,2025-11-17 00:00:00+00:00,1.60,0.0,10.188700,327.994659,Gothenburg,112672,57.660000,12.000000,5.0,5.0,NaN,NaN,4.613566,Bågskyttegatan,Sweden,1
6,2025-11-17 00:00:00+00:00,1.60,0.0,10.188700,327.994659,Majorna-Linné,60541,57.696000,11.950000,24.0,24.0,NaN,NaN,63.977062,Prinsgatan,Sweden,1
7,2025-11-17 00:00:00+00:00,1.60,0.0,10.188700,327.994659,Örgryte-Härlanda,65146,57.722000,12.012000,4.0,4.0,NaN,NaN,2.019471,Landerigatan,Sweden,1
8,2025-11-17 00:00:00+00:00,1.20,0.0,10.188700,327.994659,Centrum,69628,57.681718,11.970109,3.0,3.0,NaN,NaN,1.646546,yster Estrids Gata,Sweden,1
9,2025-11-17 00:00:00+00:00,1.55,0.0,10.188700,327.994659,Majorna-Linné,60535,57.692000,11.958000,2.0,2.0,NaN,NaN,2.179041,Annedal,Sweden,1


In [34]:
air_quality_df = air_quality_fg.read()
air_quality_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.81s) 


,date,pm25,sensor_id,street,city,country,feed_url,pm25_rolling_3d,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
0,2021-04-14 00:00:00+00:00,1.33,60535,Annedal,Majorna-Linné,Sweden,https://api.waqi.info/feed/A60535/,1.373333,0.90,1.29,1.93
1,2025-09-12 00:00:00+00:00,1.80,69724,Doktor Westrings Gata,Centrum,Sweden,https://api.waqi.info/feed/A69724/,6.193333,4.09,7.21,7.28
2,2024-11-29 00:00:00+00:00,1.23,60853,Masthugget,Majorna-Linné,Sweden,https://api.waqi.info/feed/A60853/,1.076667,0.50,0.63,2.10
3,2021-04-09 00:00:00+00:00,3.50,154549,Järnbrottsgatan,Västra Göteborg,Sweden,https://api.waqi.info/feed/A154549/,0.983333,1.33,0.80,0.82
4,2020-06-12 00:00:00+00:00,1.30,60541,Prinsgatan,Majorna-Linné,Sweden,https://api.waqi.info/feed/A60541/,1.676667,0.93,1.90,2.20
...,...,...,...,...,...,...,...,...,...,...,...
28210,2025-11-16 00:00:00+00:00,4.00,61714,Nyhemsgatan,Norra Hisingen,Sweden,https://api.waqi.info/feed/A61714/,NaN,NaN,NaN,NaN
28211,2025-11-16 00:00:00+00:00,3.00,404209,Högkullevägen,Lindome,Sweden,https://api.waqi.info/feed/A404209/,NaN,NaN,NaN,NaN
28212,2025-11-16 00:00:00+00:00,3.00,194215,Norra Sävviksvägen,Torslanda,Sweden,https://api.waqi.info/feed/A194215/,NaN,NaN,NaN,NaN
28213,2025-11-16 00:00:00+00:00,3.00,69628,yster Estrids Gata,Centrum,Sweden,https://api.waqi.info/feed/A69628/,NaN,NaN,NaN,NaN


In [ ]:
outcome_df = air_quality_df[['date', 'sensor_id', 'pm25']].copy()
preds_df = monitoring_df[['date', 'sensor_id', 'predicted_pm25']].copy()

hindcast_df = pd.merge(preds_df, outcome_df, on=["date", "sensor_id"]).sort_values(['sensor_id', 'date'])

if len(hindcast_df) == 0:
    air_quality_df_copy = air_quality_df.copy()
    air_quality_df_copy['date'] = pd.to_datetime(air_quality_df_copy['date']).dt.tz_localize(None)
    dates_with_pm25 = air_quality_df_copy[air_quality_df_copy['pm25'].notna()].sort_values('date').tail(10)
    
    date_start = dates_with_pm25['date'].min() - datetime.timedelta(days=4)
    weather_df = weather_fg.filter((weather_fg.date >= date_start) & (weather_fg.date <= dates_with_pm25['date'].max())).read()
    weather_df['date'] = pd.to_datetime(weather_df['date']).dt.tz_localize(None)
    historical_pm25 = air_quality_df_copy[(air_quality_df_copy['date'] >= date_start) & 
                                          (air_quality_df_copy['date'] <= dates_with_pm25['date'].max())][['date', 'sensor_id', 'pm25']]
    
    all_features = []
    for sensor_id, location in locations.items():
        sensor_weather = weather_df[(weather_df['sensor_id'] == sensor_id) & 
                                    (weather_df['date'].isin(dates_with_pm25[dates_with_pm25['sensor_id'] == sensor_id]['date']))].copy()
        sensor_pm25 = historical_pm25[historical_pm25['sensor_id'] == sensor_id] if not historical_pm25.empty else pd.DataFrame()
        if not sensor_weather.empty:
            sensor_weather = predict_for_sensor(sensor_weather, sensor_pm25, location, retrieved_xgboost_model, model_feature_names)
            sensor_weather['days_before_forecast_day'] = 1
            sensor_weather = sensor_weather.merge(
                dates_with_pm25[dates_with_pm25['sensor_id'] == sensor_id][['date', 'pm25']],
                on='date', how='left'
            )
            all_features.append(sensor_weather)
    
    features_df = pd.concat(all_features, ignore_index=True) if all_features else pd.DataFrame()
    hindcast_df = features_df[features_df['pm25'].notna()][['date', 'sensor_id', 'predicted_pm25', 'pm25', 'street', 'country', 'days_before_forecast_day']].copy()
    if not features_df.empty:
        monitor_fg.insert(features_df.drop('pm25', axis=1, errors='ignore'), write_options={"wait_for_job": True})
hindcast_df

,date,sensor_id,predicted_pm25,pm25
0,2025-11-16 00:00:00+00:00,154549,-1.450913,4.0
1,2025-11-16 00:00:00+00:00,59095,-0.364855,2.0
8,2025-11-16 00:00:00+00:00,59893,2.662153,3.0
6,2025-11-16 00:00:00+00:00,60535,-0.209882,2.0
7,2025-11-16 00:00:00+00:00,60541,-1.096113,24.0
4,2025-11-16 00:00:00+00:00,65146,-0.446444,4.0
3,2025-11-16 00:00:00+00:00,69628,-2.035951,3.0
2,2025-11-16 00:00:00+00:00,69724,-1.674553,6.0
5,2025-11-16 00:00:00+00:00,79750,-3.409187,1.0
9,2025-11-16 00:00:00+00:00,88372,-2.962898,3.0


### Plot the Hindcast comparing predicted with forecasted values (1-day prior forecast)

In [37]:
# Generate hindcast plots for each sensor
hindcast_file_paths = []
for sensor_id, location in locations.items():
    sensor_hindcast = hindcast_df[hindcast_df["sensor_id"] == sensor_id].copy()
    if sensor_hindcast.empty:
        continue
    
    city = location['city']
    street = location['street']
    hindcast_file_path = f"{root_dir}/models/air_quality_complete/images/hindcast_1day_{city}_{street}.png"
    
    plt = util.plot_air_quality_forecast(city, street, sensor_hindcast, hindcast_file_path, hindcast=True)
    plt.close()
    hindcast_file_paths.append(hindcast_file_path)

### Upload the prediction and hindcast plots to Hopsworks


In [40]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/airquality") == False:
    dataset_api.mkdir("Resources/airquality")

# Upload all prediction and hindcast images
for sensor_id, location in locations.items():
    city = location['city']
    street = location['street']
    
    pred_path = f"{root_dir}/models/air_quality_complete/images/forecast_{city}_{street}.png"
    hindcast_path = f"{root_dir}/models/air_quality_complete/images/hindcast_1day_{city}_{street}.png"
    
    if os.path.exists(pred_path):
        dataset_api.upload(pred_path, f"Resources/airquality/{city}_{street}_{str_today}_forecast", overwrite=True)
    if os.path.exists(hindcast_path):
        dataset_api.upload(hindcast_path, f"Resources/airquality/{city}_{street}_{str_today}_hindcast", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/airquality")

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/forecast_Västr…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/hindcast_1day_…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/forecast_Major…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/hindcast_1day_…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/forecast_Mölnd…

...

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/forecast_Lundb…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/hindcast_1day_…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/forecast_Centr…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/air_quality_complete/images/hindcast_1day_…

See images in Hopsworks here: https://c.app.hopsworks.ai:443/p/1279179/settings/fb/path/Resources/airquality
